In [ ]:
## Extracting the data and saving it as a CSV

In [2]:
import sys
sys.path.append('../sentiment_analysis')

# This is another way of doing the above, not sure which is better
sys.path.insert(0, '../mining')

from nltk import tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from vader_sentiment_analysis import SentimentAnalyser
import pandas as pd

import connector
import query

/home/alexander/anaconda/lib/python2.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
df = pd.read_csv("../data/SELECT_ALL_FROM_all_messages.csv", header = 0)

/home/sophie/anaconda3/envs/py27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
len(df)

1354178

In [16]:
list(df)

['Unnamed: 0',
 'user_id',
 'contact_id',
 'relationship',
 'channel',
 'sent_at',
 'message']

In [8]:
df_meta = pd.read_csv("../data/SELECT_ALL_FROM_all_messages_metadata.csv", header = 0)

In [9]:
len(df_meta)

1070395

In [4]:
# Not able to read this in. 
#df = pd.read_csv("../data/all_messages_cleaned.csv", encoding='utf8', engine='python', usecols = ['user_id'], header = 0)

In [10]:
df = df.drop(['Unnamed: 0'],axis=1)

In [2]:
import psycopg2
from sshtunnel import SSHTunnelForwarder

port = 5432
remote_bind_address = ('localhost', 5432)
local_bind_address = ('localhost', 5432)
user_name = 'analytics'
dbname = 'dreikanter_production'
remote_host = 'dreikanter.production.devguru.co'
remote_port = 22
conn = None
server = None


params = {
            'database': 'dreikanter_production',
            'user': 'analytics',
            'host': 'localhost',
            'port': 5432
        }

server = SSHTunnelForwarder((remote_host, remote_port),ssh_username=user_name,
                                    local_bind_address=local_bind_address,
                                    remote_bind_address=remote_bind_address)
server.start()
print "We are in!"

#"dbname=dreikanter_production user=analytics host = localhost port = 5432"
conn = psycopg2.connect(**params)

We are in!


In [20]:
conn.close()
server.stop()

In [3]:
sql_query = """SELECT * FROM all_msgs_tf;"""

dbconnection = connector.ConnectDB()
conn = dbconnection.get_connection()
q = query.Query(conn, sql_query)
query_list = q.get_query_list()
query_df = q.get_query_dataframe()
#q.write_df_to_csv()

#query_df['body'][6:16]
#query_df.head(22)

2016-08-16 09:55:59,592 | ERROR   | Couldn't open tunnel localhost:5432 <> localhost:5432 might be in use or destination not reachable.


In [81]:
query_df.shape

(2859488, 7)

In [94]:
df_no_email = query_df[query_df['channel'] != 'gmail']

In [83]:
df_no_email.shape
#print query_df[query_df.message.str.contains("Land Rover")]

(2629679, 7)

In [117]:
df_no_email = df_no_email[df_no_email['message'] != '        ']

In [118]:
df_no_email.shape

(2517804, 7)

In [119]:
b = [len(str(x).split()) for x in df_no_email['message']]

In [125]:
df_no_email['word_count'] = b

In [167]:
df_no_email[''].isnull().sum()

292013

In [4]:
df_no_email[df_no_email['positive'].isnull()]

NameError: name 'df_no_email' is not defined

In [130]:
sentiment_analyser = SentimentAnalyser()
result = sentiment_analyser.run_vader(df_no_email, 'message')

In [175]:
result.head()

,user_id,contact_id,relationship,channel,sent_at,message,to_from,word_count,positive,negative,neutral,compound
31,10,35,Ex,text_message,2011-02-18 06:50:35,Ill be there in 3,False,5,0.106,0.243,0.652,-0.1725
32,10,35,Ex,text_message,2011-02-18 06:50:47,hey I'm in the theater,False,5,0.280,0.164,0.556,0.5788
35,10,35,Ex,text_message,2011-02-22 02:22:54,:),False,1,0.727,0.000,0.273,0.7906
37,10,35,Ex,text_message,2011-03-04 07:08:49,Hey! How was the trip?,False,5,0.480,0.000,0.520,0.9399
38,10,35,Ex,text_message,2011-03-04 08:17:59,"great! still feels like I'm recovering, though!",False,7,0.129,0.000,0.871,0.5487


In [3]:
result.to_csv('all_messages_no_gmail_tf_wc_sentiment.csv', encoding='utf-8', index=False)

NameError: name 'result' is not defined

In [4]:
df = pd.read_csv('./all_messages_no_gmail_tf_wc_sentiment.csv', header=0)

/home/alexander/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (0,1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [5]:
df.head()

,user_id,contact_id,relationship,channel,sent_at,message,to_from,word_count,positive,negative,neutral,compound
0,10,35,Ex,text_message,2011-02-18 06:50:35,Ill be there in 3,False,5,0.106,0.243,0.652,-0.1725
1,10,35,Ex,text_message,2011-02-18 06:50:47,hey I'm in the theater,False,5,0.280,0.164,0.556,0.5788
2,10,35,Ex,text_message,2011-02-22 02:22:54,:),False,1,0.727,0.000,0.273,0.7906
3,10,35,Ex,text_message,2011-03-04 07:08:49,Hey! How was the trip?,False,5,0.480,0.000,0.520,0.9399
4,10,35,Ex,text_message,2011-03-04 08:17:59,"great! still feels like I'm recovering, though!",False,7,0.129,0.000,0.871,0.5487


In [ ]:
df['message'] = str(df['message'])

In [31]:
first_10 = df[1:10]
first_10 = first_10.drop(['positive'],axis=1)
first_10 = first_10.drop(['negative'],axis=1)
first_10 = first_10.drop(['neutral'],axis=1)
first_10 = first_10.drop(['compound'],axis=1)

In [10]:
sentiment_analyser = SentimentAnalyser()

In [32]:
check_10 = sentiment_analyser.run_vader(first_10, 'message')

In [33]:
check_10

,user_id,contact_id,relationship,channel,sent_at,message,to_from,word_count,positive,negative,neutral,compound
1,10,35,Ex,text_message,2011-02-18 06:50:47,hey I'm in the theater,False,5,1.000,0,0.000,0.4588
2,10,35,Ex,text_message,2011-02-22 02:22:54,:),False,1,0.000,0,1.000,0.0000
3,10,35,Ex,text_message,2011-03-04 07:08:49,Hey! How was the trip?,False,5,0.590,0,0.410,0.8011
4,10,35,Ex,text_message,2011-03-04 08:17:59,"great! still feels like I'm recovering, though!",False,7,0.448,0,0.552,0.7574
5,10,35,Ex,text_message,2011-03-04 08:26:09,"Awesome! Well, let's hang out when you return ...",False,10,0.259,0,0.741,0.5983
6,10,35,Ex,text_message,2011-03-04 09:44:44,definitely! don't spend toooo much time catalo...,False,16,0.076,0,0.924,0.2263
7,10,35,Ex,text_message,2011-03-04 09:53:23,"Oh, there's something else altogether- I'll em...",False,24,0.244,0,0.756,0.6369
8,10,35,Ex,text_message,2011-03-13 01:16:55,"Hey, I'm finally on break and would love to ha...",False,14,0.262,0,0.738,0.4926
9,10,35,Ex,text_message,2011-03-13 01:28:11,hey! i was just thinking about you. when's goo...,False,11,NaN,NaN,NaN,NaN


In [21]:
df['message'][10019]

'i zacz\xc4\x85\xc5\x82 za mn\xc4\x85 i\xc5\x9b\xc4\x87'

In [22]:
sentiment_analyser.ss.polarity_scores(df['message'][10019])

{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}

In [30]:
"☺"

'\xe2\x98\xba'

In [44]:
sentiment_analyser.ss.polarity_scores(u"\u263A")

{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}

In [35]:
sentiment_analyser.ss.polarity_scores("great! still feels like I'm recovering, though!")

{'compound': 0.8011, 'neg': 0.0, 'neu': 0.41, 'pos': 0.59}

In [ ]:
for i rows in df.iterrows()